# pips

In [1]:
!pip install requests beautifulsoup4



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install LinkedinProfile

ERROR: Could not find a version that satisfies the requirement LinkedinProfile (from versions: none)
ERROR: No matching distribution found for LinkedinProfile

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install trafilatura



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


# codigo

In [29]:
import pandas as pd
import trafilatura

# Caminho para o arquivo CSV
file_path = './database/finance_news.csv'

# Carrega o arquivo CSV em um DataFrame
df = pd.read_csv(file_path)

def extrair_texto_principal(url):
    try:
        # Baixar e extrair o texto principal da URL
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            texto_principal = trafilatura.extract(downloaded, output_format='txt')
            if texto_principal:
                return texto_principal
            else:
                return "Não foi possível encontrar o conteúdo principal."
        else:
            return "Erro ao baixar o conteúdo da página."
    except Exception as e:
        return f"Erro ao extrair texto de {url}: {e}"

# Extrai a coluna 'links' e 'title' e converte para listas
urls = df['link'].tolist()
titles = df['title'].tolist()

# Lista para armazenar os dados da nova tabela
data = []

# Extrai o texto principal e cria as entradas para a nova tabela
for title, url in zip(titles, urls):
    texto = f'{title}\n{extrair_texto_principal(url)}'  # Extrai o texto principal da URL
    data.append({'news_text': texto})

# Cria um novo DataFrame com os dados processados
df_final = pd.DataFrame(data, columns=['news_text'])

# Exibe a tabela final
print(df_final)

# Salvar a tabela em um novo arquivo CSV, se necessário
df_final.to_csv('./database/news_texts.csv', index=False)

                                             news_text
0    Shiba Inu Investors Now Hold This New Cryptocu...
1    Cautious car buyers blamed for 10 per cent dro...
2    NMFTA’s Cybersecurity Team Continues to Showca...
3    Ethereum (ETH) Whales Grab New 1000x Memecoin ...
4    A 'soft landing' is still on the table, econom...
..                                                 ...
167  Martin Lewis warns drivers who've received £3,...
168  Heads up homeowners - Households set for cash ...
169  Money blog: Wizz Air launches subscription wit...
170  Martin Lewis' MSE online letter that has saved...
171  Money blog: Inflation rises for first time sin...

[172 rows x 1 columns]
